In [ ]:
from datagen.chordgen import generate_sine_chords
from datagen.fxgen import batch_process_samples
from pathlib import Path

In [ ]:
# Set up directory paths
midi = Path("data/midi")
sine_dir = Path("data/sine")
reverb_dir = Path("data/fx")
noise_dir = Path("data/noise")

In [ ]:
#generate sine chord dataset - this is the control dataset 
generate_sine_chords()

#process all sine chords with randomized reverb
batch_process_samples(sinechord)

#process all sine chords with randomized noise
batch_process_samples(sinechord)

In [ ]:
#import pretrained model

#run model on sine chords

#run model on reverb chords

#run model on noise chords

In [ ]:
# visualize results